# Extrac basic info for NN input
There are required Basic info:
X, S, mask, lengths, chain_M, residue_idx, mask_self, chain_encoding_all = featurize(batch, device)<br>
As they will be provided as inputs for model: <br>
log_probs = model(X, S, mask, chain_M, residue_idx, chain_encoding_all) <br>

Inputs: batch containig  # protein (seq, chain_seq, cooridinates, info about whether chains are masked)

X: [Batch, L_max, 4, 3] # L_max (longest seqence of all proteins in the batch, 4: N CA C O, 3: x, y, z coordinates)   
S: ground truth, sequence displayed as int for example A as 0, C as 1  
mask: mask for padding, padding as 0  
lengths: np array of seq_length for protein in the batch  
chain_M: [Batch, L_max] 1 for masked 0 for not  
residue_idx: [Batch, l] l: length of seqence, encode in hopping way for differnt chain  
mask_self: [Batch, L_max, L_max]: residue pairwise info: within chain as 0, inter chain as 1  
chain_encoding_all: [Batch, L_max] chain code as 1 1 1 1 1 ...2 2 2... 3 3 3...
'''

In [3]:
# example of batch inputs

import numpy as np

batch = [
    {
        'seq_chain_A': 'MKLVFLVLLVFVQGF',
        'coords_chain_A': {'N_chain_A': np.random.rand(16, 3), 'CA_chain_A': np.random.rand(16, 3), 'C_chain_A': np.random.rand(16, 3), 'O_chain_A': np.random.rand(16, 3)},
        'seq_chain_B': 'MSVKVEEVG',
        'coords_chain_B': {'N_chain_B': np.random.rand(9, 3), 'CA_chain_B': np.random.rand(9, 3), 'C_chain_B': np.random.rand(9, 3), 'O_chain_B': np.random.rand(9, 3)},
        'seq_chain_C': 'ATCGATCGATCGATCG',
        'coords_chain_C': {'N_chain_C': np.random.rand(16, 3), 'CA_chain_C': np.random.rand(16, 3), 'C_chain_C': np.random.rand(16, 3), 'O_chain_C': np.random.rand(16, 3)},
        'masked_list': ['A', 'B'],
        'visible_list': ['C'],
        'num_of_chains': 3,
        'seq': 'MKLVFLVLLVFVQGF'+ 'MSVKVEEVG' + 'ATCGATCGATCGATCG'
    },
      {
        'seq_chain_X': 'ACDEFGHIKLMNPQRSTVWY',
        'coords_chain_X': {'N_chain_X': np.random.rand(20, 3), 'CA_chain_X': np.random.rand(20, 3), 'C_chain_X': np.random.rand(20, 3), 'O_chain_X': np.random.rand(20, 3)},
        'seq_chain_Y': 'ABCDEFGHIJKLM',
        'coords_chain_Y': {'N_chain_Y': np.random.rand(13, 3), 'CA_chain_Y': np.random.rand(13, 3), 'C_chain_Y': np.random.rand(13, 3), 'O_chain_Y': np.random.rand(13, 3)},
        'seq_chain_Z': 'JKLMNOPQRST',
        'coords_chain_Z': {'N_chain_Z': np.random.rand(11, 3), 'CA_chain_Z': np.random.rand(11, 3), 'C_chain_Z': np.random.rand(11, 3), 'O_chain_Z': np.random.rand(11, 3)},
        'masked_list': ['X', 'Y'],
        'visible_list': ['Z'],
        'num_of_chains': 3,
        'seq': 'ACDEFGHIKLMNPQRSTVWY'+'ABCDEFGHIJKLM'+'JKLMNOPQRST'

    }
]

In [4]:
#original Featurize function
def featurize(batch, device):
    alphabet = 'ACDEFGHIKLMNPQRSTVWYX'
    B = len(batch)
    lengths = np.array([len(b['seq']) for b in batch], dtype=np.int32) #sum of chain seq lengths
    L_max = max([len(b['seq']) for b in batch])
    X = np.zeros([B, L_max, 4, 3])
    residue_idx = -100*np.ones([B, L_max], dtype=np.int32) #residue idx with jumps across chains
    chain_M = np.zeros([B, L_max], dtype=np.int32) #1.0 for the bits that need to be predicted, 0.0 for the bits that are given
    mask_self = np.ones([B, L_max, L_max], dtype=np.int32) #for interface loss calculation - 0.0 for self interaction, 1.0 for other
    chain_encoding_all = np.zeros([B, L_max], dtype=np.int32) #integer encoding for chains 0, 0, 0,...0, 1, 1,..., 1, 2, 2, 2...
    S = np.zeros([B, L_max], dtype=np.int32) #sequence AAs integers
    init_alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G','H', 'I', 'J','K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T','U', 'V','W','X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g','h', 'i', 'j','k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't','u', 'v','w','x', 'y', 'z']
    extra_alphabet = [str(item) for item in list(np.arange(300))]
    chain_letters = init_alphabet + extra_alphabet
    for i, b in enumerate(batch):
        masked_chains = b['masked_list']
        visible_chains = b['visible_list']
        all_chains = masked_chains + visible_chains
        visible_temp_dict = {}
        masked_temp_dict = {}
        for step, letter in enumerate(all_chains):
            chain_seq = b[f'seq_chain_{letter}']
            if letter in visible_chains:
                visible_temp_dict[letter] = chain_seq
            elif letter in masked_chains:
                masked_temp_dict[letter] = chain_seq
        for km, vm in masked_temp_dict.items():
            for kv, vv in visible_temp_dict.items():
                if vm == vv:
                    if kv not in masked_chains:
                        masked_chains.append(kv)
                    if kv in visible_chains:
                        visible_chains.remove(kv)
        all_chains = masked_chains + visible_chains
        random.shuffle(all_chains) #randomly shuffle chain order
        num_chains = b['num_of_chains']
        mask_dict = {}
        x_chain_list = []
        chain_mask_list = []
        chain_seq_list = []
        chain_encoding_list = []
        c = 1
        l0 = 0
        l1 = 0
        for step, letter in enumerate(all_chains):
            if letter in visible_chains:
                chain_seq = b[f'seq_chain_{letter}']
                chain_length = len(chain_seq)
                chain_coords = b[f'coords_chain_{letter}'] #this is a dictionary
                chain_mask = np.zeros(chain_length) #0.0 for visible chains
                x_chain = np.stack([chain_coords[c] for c in [f'N_chain_{letter}', f'CA_chain_{letter}', f'C_chain_{letter}', f'O_chain_{letter}']], 1) #[chain_length,4,3]
                x_chain_list.append(x_chain)
                chain_mask_list.append(chain_mask)
                chain_seq_list.append(chain_seq)
                chain_encoding_list.append(c*np.ones(np.array(chain_mask).shape[0])) # decode chain index for each residue
                l1 += chain_length
                mask_self[i, l0:l1, l0:l1] = np.zeros([chain_length, chain_length])
                residue_idx[i, l0:l1] = 100*(c-1)+np.arange(l0, l1)
                l0 += chain_length
                c+=1
            elif letter in masked_chains: 
                chain_seq = b[f'seq_chain_{letter}']
                chain_length = len(chain_seq)
                chain_coords = b[f'coords_chain_{letter}'] #this is a dictionary
                chain_mask = np.ones(chain_length) #0.0 for visible chains
                x_chain = np.stack([chain_coords[c] for c in [f'N_chain_{letter}', f'CA_chain_{letter}', f'C_chain_{letter}', f'O_chain_{letter}']], 1) #[chain_lenght,4,3]
                x_chain_list.append(x_chain)
                chain_mask_list.append(chain_mask)
                chain_seq_list.append(chain_seq)
                chain_encoding_list.append(c*np.ones(np.array(chain_mask).shape[0]))
                l1 += chain_length
                mask_self[i, l0:l1, l0:l1] = np.zeros([chain_length, chain_length])
                residue_idx[i, l0:l1] = 100*(c-1)+np.arange(l0, l1)
                l0 += chain_length
                c+=1
        x = np.concatenate(x_chain_list,0) #[L, 4, 3]
        all_sequence = "".join(chain_seq_list)
        m = np.concatenate(chain_mask_list,0) #[L,], 1.0 for places that need to be predicted
        chain_encoding = np.concatenate(chain_encoding_list,0)

        l = len(all_sequence)
        x_pad = np.pad(x, [[0,L_max-l], [0,0], [0,0]], 'constant', constant_values=(np.nan, ))
        X[i,:,:,:] = x_pad

        m_pad = np.pad(m, [[0,L_max-l]], 'constant', constant_values=(0.0, ))
        chain_M[i,:] = m_pad

        chain_encoding_pad = np.pad(chain_encoding, [[0,L_max-l]], 'constant', constant_values=(0.0, ))
        chain_encoding_all[i,:] = chain_encoding_pad

        # Convert to labels
        indices = np.asarray([alphabet.index(a) for a in all_sequence], dtype=np.int32)
        S[i, :l] = indices

    isnan = np.isnan(X)
    mask = np.isfinite(np.sum(X,(2,3))).astype(np.float32) # if cooridnate contain Nan or infinite, set it as 1 as marked and need to be predicted
    X[isnan] = 0. # convert all Nan to zero in cooridnates 

    # Conversion
    residue_idx = torch.from_numpy(residue_idx).to(dtype=torch.long,device=device)
    S = torch.from_numpy(S).to(dtype=torch.long,device=device)
    X = torch.from_numpy(X).to(dtype=torch.float32, device=device)
    mask = torch.from_numpy(mask).to(dtype=torch.float32, device=device)
    mask_self = torch.from_numpy(mask_self).to(dtype=torch.float32, device=device)
    chain_M = torch.from_numpy(chain_M).to(dtype=torch.float32, device=device)
    chain_encoding_all = torch.from_numpy(chain_encoding_all).to(dtype=torch.long, device=device)
    return X, S, mask, lengths, chain_M, residue_idx, mask_self, chain_encoding_all

In [5]:
#breakdown of featurize function
#Part I. initialization placeholder with right dimension
alphabet = 'ACDEFGHIKLMNPQRSTVWYX'
B = len(batch)  #2
lengths = np.array([len(b['seq']) for b in batch], dtype=np.int32) #sum of chain seq lengths
L_max = max([len(b['seq']) for b in batch]) # longest sequence
X = np.zeros([B, L_max, 4, 3]) # initialize X with zero, for store cooridnates
residue_idx = -100*np.ones([B, L_max], dtype=np.int32) #residue idx with jumps across chains, initialzie with -100
chain_M = np.zeros([B, L_max], dtype=np.int32) #1.0 for the bits that need to be predicted, 0.0 for the bits that are given
mask_self = np.ones([B, L_max, L_max], dtype=np.int32) #for interface loss calculation - 0.0 for self interaction, 1.0 for other
chain_encoding_all = np.zeros([B, L_max], dtype=np.int32) #integer encoding for chains 0, 0, 0,...0, 1, 1,..., 1, 2, 2, 2...
S = np.zeros([B, L_max], dtype=np.int32) #sequence AAs integers
init_alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G','H', 'I', 'J','K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T','U', 'V','W','X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g','h', 'i', 'j','k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't','u', 'v','w','x', 'y', 'z']
extra_alphabet = [str(item) for item in list(np.arange(300))]
chain_letters = init_alphabet + extra_alphabet


## Part II: fill placeholder with basic protein info uisng looped for loops

pseudoCode:
For each item in batch (indexed by i):

### Initialize lists for chains:

masked_chains = list of chains to be masked from the batch.
visible_chains = list of visible chains from the batch.
all_chains = concatenation of masked_chains and visible_chains.

### mapping seq to chain
Prepare temporary dictionaries:
visible_temp_dict = map visible chains to their sequences.
masked_temp_dict = map masked chains to their sequences.


For each chain:
Get its sequence chain_seq.
If it is in visible_chains, add it to visible_temp_dict.
If it is in masked_chains, add it to masked_temp_dict.

### check contaminatin between maksed and visible chains, 
Match masked and visible chains by sequence: (to make sure on contamination between test and train)

For each masked chain and each visible chain:
If their sequences match:
Add the visible chain to masked_chains if not already present. 
Remove the visible chain from visible_chains.
Shuffle all chains randomly to create a random chain order.

### start chain processing
Initialize variables for chain processing:

mask_dict, x_chain_list, chain_mask_list, chain_seq_list, and chain_encoding_list for storage.
Counters: c = 1, l0 = 0, l1 = 0.
Iterate over all_chains:

For each chain:
Retrieve chain_seq, chain_length, and chain_coords.
Create chain_mask:
0s for visible chains.
1s for masked chains.
Stack coordinates for atoms (N, CA, C, O) into x_chain.
Append data (x_chain, chain_mask, chain_seq, chain_encoding) to their respective lists.
Update:
mask_self to store self-interaction mask for the chain.
residue_idx to store residue indices for the chain.
Counters: l0, l1, and c.
Concatenate processed chain data:

x = concatenation of all x_chain ([L, 4, 3]).
all_sequence = concatenation of all chain sequences.
m = concatenation of all chain_mask ([L]).
chain_encoding = concatenation of all chain encodings.

### padding to L_max
Handle padding:

Pad x, m, and chain_encoding to the maximum length L_max with default values.
Store padded data:

Assign padded x, m, and chain_encoding to arrays X[i], chain_M[i], and chain_encoding_all[i].
Convert sequences to labels:

Map characters in all_sequence to indices based on alphabet.
Store the sequence indices in S[i].

In [6]:
b=batch[0]
masked_chains = b['masked_list']
visible_chains = b['visible_list']
all_chains = masked_chains + visible_chains
visible_temp_dict = {}
masked_temp_dict = {}
for step, letter in enumerate(all_chains):
    chain_seq = b[f'seq_chain_{letter}']
    if letter in visible_chains:
        visible_temp_dict[letter] = chain_seq
    elif letter in masked_chains:
        masked_temp_dict[letter] = chain_seq
for km, vm in masked_temp_dict.items():
    for kv, vv in visible_temp_dict.items():
        if vm == vv:
            if kv not in masked_chains:
                masked_chains.append(kv)
            if kv in visible_chains:
                visible_chains.remove(kv)
all_chains = masked_chains + visible_chains

print(all_chains)


['A', 'B', 'C']


In [7]:

num_chains = b['num_of_chains']
mask_dict = {}
x_chain_list = []
chain_mask_list = []
chain_seq_list = []
chain_encoding_list = []
c = 1
l0 = 0
l1 = 0
for step, letter in enumerate(all_chains):
    if letter in visible_chains:
        chain_seq = b[f'seq_chain_{letter}']
        chain_length = len(chain_seq)
        chain_coords = b[f'coords_chain_{letter}'] #this is a dictionary
        chain_mask = np.zeros(chain_length) #0.0 for visible chains
        x_chain = np.stack([chain_coords[c] for c in [f'N_chain_{letter}', f'CA_chain_{letter}', f'C_chain_{letter}', f'O_chain_{letter}']], 1) #[chain_length,4,3]
        x_chain_list.append(x_chain)
        chain_mask_list.append(chain_mask)
        chain_seq_list.append(chain_seq)
        chain_encoding_list.append(c*np.ones(np.array(chain_mask).shape[0])) # decode chain index for each residue
        l1 += chain_length
        mask_self[0, l0:l1, l0:l1] = np.zeros([chain_length, chain_length])
        residue_idx[0, l0:l1] = 100*(c-1)+np.arange(l0, l1)
        l0 += chain_length
        c+=1
    elif letter in masked_chains: 
        chain_seq = b[f'seq_chain_{letter}']
        chain_length = len(chain_seq)
        chain_coords = b[f'coords_chain_{letter}'] #this is a dictionary
        chain_mask = np.ones(chain_length) #0.0 for visible chains
        x_chain = np.stack([chain_coords[c] for c in [f'N_chain_{letter}', f'CA_chain_{letter}', f'C_chain_{letter}', f'O_chain_{letter}']], 1) #[chain_lenght,4,3]
        x_chain_list.append(x_chain)
        chain_mask_list.append(chain_mask)
        chain_seq_list.append(chain_seq)
        chain_encoding_list.append(c*np.ones(np.array(chain_mask).shape[0]))
        l1 += chain_length
        mask_self[0, l0:l1, l0:l1] = np.zeros([chain_length, chain_length])
        residue_idx[0, l0:l1] = 100*(c-1)+np.arange(l0, l1)
        l0 += chain_length
        c+=1

print(len(x_chain_list))
print(chain_mask_list)
print(chain_seq_list)
print(chain_encoding_list)


3
[array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), array([1., 1., 1., 1., 1., 1., 1., 1., 1.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])]
['MKLVFLVLLVFVQGF', 'MSVKVEEVG', 'ATCGATCGATCGATCG']
[array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), array([2., 2., 2., 2., 2., 2., 2., 2., 2.]), array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.])]


In [ ]:
x = np.concatenate(x_chain_list,0) #[L, 4, 3]
print(x.shape)
all_sequence = "".join(chain_seq_list)
m = np.concatenate(chain_mask_list,0) #[L,], 1.0 for places that need to be predicted
chain_encoding = np.concatenate(chain_encoding_list,0)

l = len(all_sequence)
print(l)
print(L_max)
x_pad = np.pad(x, [[0,L_max-l-1], [0,0], [0,0]], 'constant', constant_values=(np.nan, ))
print(x_pad.shape)
X[0,:,:,:] = x_pad

m_pad = np.pad(m, [[0,L_max-l]], 'constant', constant_values=(0.0, ))
chain_M[0,:] = m_pad

chain_encoding_pad = np.pad(chain_encoding, [[0,L_max-l]], 'constant', constant_values=(0.0, ))
chain_encoding_all[0,:] = chain_encoding_pad

# Convert to labels
indices = np.asarray([alphabet.index(a) for a in all_sequence], dtype=np.int32)
S[0, :l] = indices

isnan = np.isnan(X)
mask = np.isfinite(np.sum(X,(2,3))).astype(np.float32) # if cooridnate contain Nan or infinite, set it as 1 as marked and need to be predicted
X[isnan] = 0. # convert all Nan to zero in cooridnates 

print(X[0,:3,:,:])
print(S[0,:]) # 0 corresponds to both A and padding, better if they are differnt
print(mask)  # 1 for real xyz , 0 for padding
print(chain_M[0,:]) # 1 for masked residue

(41, 4, 3)
40
44
(44, 4, 3)
[[[0.06354426 0.08509534 0.85666013]
  [0.9364399  0.57632678 0.74632588]
  [0.43980392 0.09177521 0.7680629 ]
  [0.94961742 0.50599196 0.19437512]]

 [[0.60256077 0.50263063 0.53908247]
  [0.10484899 0.89915181 0.66226552]
  [0.90085038 0.30532721 0.29488071]
  [0.61960395 0.5194988  0.56602763]]

 [[0.42652207 0.03456161 0.44309081]
  [0.84359014 0.91269089 0.51876517]
  [0.90151762 0.80485696 0.74073618]
  [0.42065161 0.87299604 0.63405803]]]
[10  8  9 17  4  9 17  9  9 17  4 17 13  5  4 10 15 17  8 17  3  3 17  5
  0 16  1  5  0 16  1  5  0 16  1  5  0 16  1  5  0  0  0  0]
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]


In [ ]:
[alphabet[i] for i in S[0,:]]
['MKLVFLVLLVFVQGF', 'MSVKVEEVG', 'ATCGATCGATCGATCG']

['M',
 'K',
 'L',
 'V',
 'F',
 'L',
 'V',
 'L',
 'L',
 'V',
 'F',
 'V',
 'Q',
 'G',
 'F',
 'M',
 'S',
 'V',
 'K',
 'V',
 'E',
 'E',
 'V',
 'G',
 'A',
 'T',
 'C',
 'G',
 'A',
 'T',
 'C',
 'G',
 'A',
 'T',
 'C',
 'G',
 'A',
 'T',
 'C',
 'G',
 'A',
 'A',
 'A',
 'A']